In [ ]:
import torch
print(torch.__version__)

2.3.0+cu121


In [ ]:
from transformers import AutoTokenizer, AutoModelForCausalLM
import torch
from transformers import TextStreamer, StoppingCriteria, StoppingCriteriaList

class StopAtResponseCriteria(StoppingCriteria):
    def __init__(self, tokenizer, stop_sequence):
        self.tokenizer = tokenizer
        self.stop_sequence = stop_sequence

    def __call__(self, input_ids, scores, **kwargs):
        decoded_output = self.tokenizer.decode(input_ids[0], skip_special_tokens=True)
        return self.stop_sequence in decoded_output

# 모델과 토크나이저 로드==>Google Colab A100에서는 실행가능함
#model_name = "Younggooo/Yg_senti1" ##model_name="beomi/Llama-3-Open-Ko-8B",  모델 크기가 더 큼
model_name = "Younggooo/Yg_senti2"  ### model_name = "unsloth/llama-3-8b-bnb-4bit", 모델크기가 yg_senti1보다 작음
tokenizer = AutoTokenizer.from_pretrained(model_name)

# 모델을 GPU로 직접 로드(A100으로 실행해서 성공함)
model = AutoModelForCausalLM.from_pretrained(model_name).to("cuda")

#Google Colab에서 분산 처리는 실패
# 여러 GPU 이용. 여러 장치에 모델 파라미터를 자동으로 분산시켜 로드하고,
# 필요한 경우 디스크로 일부 파라미터를 오프로딩하여 메모리를 효율적으로 관리
#with init_empty_weights():
#    model = AutoModelForCausalLM.from_pretrained(model_name, device_map="auto", offload_folder="offload")


In [ ]:
# 알파카 포맷 설정
alpaca_prompt = """Below are instructions that describe the assignment. Write a complete answer to the positive and negative sentiment analysis of the given Korean sentences.

### Instruction:
{}

### Response:
"""

# 분석할 입력 문장들
input_texts = [
    "마음에 들어요 배송 빨랐고요",
    "다른 색상 구매하고 싶어요 넘나 이뻐서",
    "다 좋은데.. 배송 느린 것만 아쉽네오ㅠㅠㅠ 가격 품질면에서는 아쉬운 점 하나도 없습니다. 역시 페리페라 틴트는 믿고 구매할 가치 충분합니다ㅋㅋ",
    "금요일에 시켰는데 토요일에 왔어요 ㅋㅋㅋ 배송 짱 빠름 립스틱 색깔도 너무 예쁘고 각질부각 안돼서 너무 좋아요",
    "30대 건성 일주일 미만 2015년 발림 좋고 오래 지속 되네요",
    " 싼맛 2개 샀는데.. 이건 뭐 색상가 완전히 틀리네 오 .. 에여 3번 색상 완전 틀림.. 매혹은 무슨... 그냥 레드 같은데 친구나 줘버려야겠음"
]

# 결과를 저장할 리스트
output_texts = []

for input_text in input_texts:
    # 입력 텍스트를 포맷팅하여 토큰화
    prompt = alpaca_prompt.format(input_text)
    inputs = tokenizer(prompt, return_tensors="pt").to("cuda")  # Move inputs to GPU

    # 모델을 사용하여 텍스트 생성
    generated_outputs = model.generate(
        **inputs,
        max_new_tokens=150,  # 최대 생성 토큰 수를 설정합니다.
        num_beams=5,  # beam search를 사용하여 더 좋은 결과를 얻도록 설정합니다.
        early_stopping=True  # 조기 중단을 사용하여 적절한 길이에서 생성을 멈추도록 설정합니다.
    )


    # 생성된 텍스트 디코딩
    output_text = tokenizer.decode(generated_outputs[0], skip_special_tokens=True)


    # Prompt 부분 제거 및 중복 방지
    if prompt in output_text:
        output_text = output_text.replace(prompt, "").strip()

    # 응답 부분 추출
    response_start = output_text.find("Sentiment:")
    if response_start != -1:
        output_text = output_text[response_start:]

    # 불필요한 출력 제거
    end_sequence = "Below are instructions that"
    end_index = output_text.find(end_sequence)
    if end_index != -1:
        output_text = output_text[:end_index].strip()


    #Fix: Removed extra indentation
    output_texts.append(output_text)

# 결과 출력
for input_text, output_text in zip(input_texts, output_texts):
    print(f"Input: {input_text}\nOutput: {output_text}\n")

Input: 마음에 들어요 배송 빨랐고요
Output: Sentiment: positive, Probability: 0.9, Positive Topics: 배송,마음, Negative Topics: Sentiment: positive, Probability: 0.9, Positive Topics: 배송,마음, Negative Topics: Sentiment: positive, Probability: 0.9

Input: 다른 색상 구매하고 싶어요 넘나 이뻐서
Output: Sentiment: positive, Probability: 0.9, Positive Topics: 색상,구매,이쁨, Negative Topics: Sentiment: positive, Probability: 0.9, Positive Topics: 색상,구매,이쁨, Negative Topics:

Input: 다 좋은데.. 배송 느린 것만 아쉽네오ㅠㅠㅠ 가격 품질면에서는 아쉬운 점 하나도 없습니다. 역시 페리페라 틴트는 믿고 구매할 가치 충분합니다ㅋㅋ
Output: Sentiment: positive, Probability: 0.9, Positive Topics: 가격,품질,페리페라, 틴트, Negative Topics: 배송Sentiment: positive, Probability: 0.9, Positive Topics: 가격,품질,페리페라, 틴트, Negative Topics: 배송

Input: 금요일에 시켰는데 토요일에 왔어요 ㅋㅋㅋ 배송 짱 빠름 립스틱 색깔도 너무 예쁘고 각질부각 안돼서 너무 좋아요
Output: Sentiment: positive, Probability: 0.95, Positive Topics: 배송,색깔,각질부각, Negative Topics:

Input: 30대 건성 일주일 미만 2015년 발림 좋고 오래 지속 되네요
Output: Sentiment: positive, Probability: 0.9, Positive Topics: 발림,오래 지속, Negat

#출력결과 ===> 으마무시하게 GPU VRAM 용량 타네 그려.ㅠㅠ
#Senti1(Llama3 8b ko) vs Senti2(Llama3 8b) 비교

##Senti1로 생성한 것(Llama3 8b ko)
Input: 마음에 들어요 배송 빨랐고요<BR>
Output: Sentiment: positive, Probability: 0.9, Positive Topics: 배송,마음, Negative Topics:

##Senti2로 생성한 것(Llama3 8b)
Input: 마음에 들어요 배송 빨랐고요<BR>
Output: Sentiment: positive, Probability: 0.85, Positive Topics: 배송, Negative Topics:

-------------------------------------------------------------------
##Senti1로 생성한 것(Llama3 8b ko)
Input: 다른 색상 구매하고 싶어요 넘나 이뻐서<BR>
Output: Sentiment: positive, Probability: 0.9, Positive Topics: 색상,이쁨, Negative Topics:

##Senti2로 생성한 것(Llama3 8b)
Input: 다른 색상 구매하고 싶어요 넘나 이뻐서<BR>
Output: Sentiment: positive, Probability: 0.95, Positive Topics: 색상,이쁨, Negative Topics:

-------------------------------------------------------------------
##Senti1로 생성한 것(Llama3 8b ko)
Input: 다 좋은데.. 배송 느린 것만 아쉽네오ㅠㅠㅠ 가격 품질면에서는 아쉬운 점 하나도 없습니다. 역시 페리페라 틴트는 믿고 구매할 가치 충분합니다ㅋㅋ<BR>
Output: Sentiment: positive, Probability: 0.9, Positive Topics: 가격,품질,페리페라, 틴트, Negative Topics: 배송

##Senti2로 생성한 것(Llama3 8b)
Input: 다 좋은데.. 배송 느린 것만 아쉽네오ㅠㅠㅠ 가격 품질면에서는 아쉬운 점 하나도 없습니다. 역시 페리페라 틴트는 믿고 구매할 가치 충분합니다ㅋㅋ<BR>
Output: Sentiment: positive, Probability: 0.8, Positive Topics: 가격,품질, Negative Topics: 배송

-------------------------------------------------------------------
##Senti1로 생성한 것(Llama3 8b ko)
Input: 금요일에 시켰는데 토요일에 왔어요 ㅋㅋㅋ 배송 짱 빠름 립스틱 색깔도 너무 예쁘고 각질부각 안돼서 너무 좋아요<BR>
Output: Sentiment: positive, Probability: 0.95, Positive Topics: 배송,색깔,각질부각, Negative Topics:

##Senti2로 생성한 것(Llama3 8b)
Input: 금요일에 시켰는데 토요일에 왔어요 ㅋㅋㅋ 배송 짱 빠름 립스틱 색깔도 너무 예쁘고 각질부각 안돼서 너무 좋아요<BR>
Output: Sentiment: positive, Probability: 0.95, Positive Topics: 배송,색깔,각질부각, Negative Topics:

-------------------------------------------------------------------
##Senti1로 생성한 것(Llama3 8b ko)
Input: 30대 건성 일주일 미만 2015년 발림 좋고 오래 지속 되네요<BR>
Output: Sentiment: positive, Probability: 0.9, Positive Topics: 발림,오래 지속, Negative Topics:

##Senti2로 생성한 것(Llama3 8b)
Input: 30대 건성 일주일 미만 2015년 발림 좋고 오래 지속 되네요<BR>
Output: Sentiment: positive, Probability: 0.9, Positive Topics: 건성,일주일,미만,발림,지속, Negative Topics:

-------------------------------------------------------------------
##Senti1로 생성한 것(Llama3 8b ko)
Input:  싼맛 2개 샀는데.. 이건 뭐 색상가 완전히 틀리네 오 .. 에여 3번 색상 완전 틀림.. 매혹은 무슨... 그냥 레드 같은데 친구나 줘버려야겠음<BR>
Output: Sentiment: negative, Probability: 0.2, Positive Topics:, Negative Topics: 색상,매혹,레드

##Senti2로 생성한 것(Llama3 8b)
Input:  싼맛 2개 샀는데.. 이건 뭐 색상가 완전히 틀리네 오 .. 에여 3번 색상 완전 틀림.. 매혹은 무슨... 그냥 레드 같은데 친구나 줘버려야겠음<BR>
Output: Sentiment: negative, Probability: 0.2, Positive Topics:, Negative Topics: 색상,매혹


In [ ]:
import torch
print(torch.__version__)

2.3.0+cu121


In [ ]:

torch.cuda.is_available()


True

In [ ]:
torch.cuda.current_device()

0